# BEDBASE workflow tutorial

This demo demonstrates how to process, analyze, visualize, and serve BED files. The process has 5 steps: First, the [bedmaker](https://github.com/databio/bedmaker) pipeline converts different region data files (bed, bedGraph, bigBed, bigWig, and wig) into BED format and generates bigBed format for each file for visualization in Genome Browser.  An optional step, the [bedqc](https://github.com/databio/bedqc) pipline, flags the BED files that you might not want to include in the downstream analysis.  Second, individual BED files are analyzed using the [bedstat](https://github.com/databio/bedstat) pipeline. Third, BED files are grouped and then analyzed as groups using the [bedbuncher](https://github.com/databio/bedbuncher) pipeline. Fourth, [bedembed](https://github.com/databio/bedembed) uses the StarSpace method to embed the bed files and the meta data, and the distances between the file labels and trained search terms will be calculated with cosine distance. Finally, the BED files, along with statistics, plots, and grouping information, is served via a web interface and RESTful API using the [bedhost](https://github.com/databio/bedhost) package.

**Glossary of terms:**

- *bedfile*: a tab-delimited file with one genomic region per line. Each genomic region is decribed by 3 required columns: chrom, start and end.
- *bedset*: a collection of BED files grouped by with a shared biological, experimental, or logical criterion.


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Preparation" data-toc-modified-id="1.-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. Preparation</a></span></li><li><span><a href="#2.-BEDMAKER:-convert-non-bed-files-into-bed-files-and-generate-bigBed-files-for-genome-browser-tracks" data-toc-modified-id="2.-BEDMAKER:-convert-non-bed-files-into-bed-files-and-generate-bigBed-files-for-genome-browser-tracks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2. BEDMAKER: convert non-bed files into bed files and generate bigBed files for genome browser tracks</a></span><ul class="toc-item"><li><span><a href="#Get-a-PEP-describing-the-files-to-process" data-toc-modified-id="Get-a-PEP-describing-the-files-to-process-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get a PEP describing the files to process</a></span></li><li><span><a href="#Run-bedmaker-on-the-demo-PEP" data-toc-modified-id="Run-bedmaker-on-the-demo-PEP-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Run bedmaker on the demo PEP</a></span></li></ul></li><li><span><a href="#OPTIONAL-BEDQC:-flag-bed-files-for-futher-evaluation-to-determine-whether-they-should-be-included-in-the-downstream-analysis" data-toc-modified-id="OPTIONAL-BEDQC:-flag-bed-files-for-futher-evaluation-to-determine-whether-they-should-be-included-in-the-downstream-analysis"><span class="toc-item-num">&nbsp;&nbsp;</span>OPTIONAL BEDQC: flag bed files for futher evaluation to determine whether they should be included in the downstream analysis</a></span><ul class="toc-item"><li><span><a href="#Get-a-PEP-describing-the-files-to-process" data-toc-modified-id="Get-a-PEP-describing-the-files-to-process"><span class="toc-item-num">&nbsp;&nbsp;</span>Get a PEP describing the files to process</a></span></li><li><span><a href="#Run-bedqc-on-the-demo-PEP" data-toc-modified-id="Run-bedqc-on-the-demo-PEP"><span class="toc-item-num">&nbsp;&nbsp;</span>Run bedqc on the demo PEP</a></span></li></ul></li><li><span><a href="#3.-BEDSTAT:-Generate-statistics-and-plots-of-BED-files" data-toc-modified-id="3.-BEDSTAT:-Generate-statistics-and-plots-of-BED-files-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. BEDSTAT: Generate statistics and plots of BED files</a></span><ul class="toc-item"><li><span><a href="#Get-a-PEP-describing-the-bedfiles-to-process" data-toc-modified-id="Get-a-PEP-describing-the-bedfiles-to-process-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get a PEP describing the bedfiles to process</a></span></li><li><span><a href="#Install-bedstat-dependencies" data-toc-modified-id="Install-bedstat-dependencies-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Install bedstat dependencies</a></span></li><li><span><a href="#Inititiate-a-local-PostgreSQL-instance" data-toc-modified-id="Inititiate-a-local-PostgreSQL-instance-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Inititiate a local PostgreSQL instance</a></span></li><li><span><a href="#Run-bedstat--on-the-demo-PEP" data-toc-modified-id="Run-bedstat--on-the-demo-PEP-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Run bedstat  on the demo PEP</a></span></li></ul></li><li><span><a href="#4.-BEDBUNCHER:-Create-bedsets-and-their-respective-statistics" data-toc-modified-id="4.-BEDBUNCHER:-Create-bedsets-and-their-respective-statistics-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4. BEDBUNCHER: Create bedsets and their respective statistics</a></span><ul class="toc-item"><li><span><a href="#Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query" data-toc-modified-id="Create-a-new-PEP-describing-the-bedset-name-and-specific-JSON-query-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create a new PEP describing the bedset name and specific JSON query</a></span></li><li><span><a href="#Create-outputs-directory-and-install-bedbuncher-CML-dependencies" data-toc-modified-id="Create-outputs-directory-and-install-bedbuncher-CML-dependencies-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Create outputs directory and install bedbuncher CML dependencies</a></span></li><li><span><a href="#Run-bedbuncher-using-Looper" data-toc-modified-id="Run-bedbuncher-using-Looper-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Run bedbuncher using Looper</a></span></li></ul></li><li><span><a href="#5.-BEDHOST:--Serve-BED-files-and-API-to-explore-pipeline-outputs" data-toc-modified-id="5.-BEDHOST:--Serve-BED-files-and-API-to-explore-pipeline-outputs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5. BEDHOST:  Serve BED files and API to explore pipeline outputs</a></span></li></ul></div>

## 1. Preparation 

First, we will create a tutorial directory where we'll store the bedbase pipelines and files to be processed. We'll also need to create an environment variable that points to the tutorial directory (we'll need this variable later). 

In [3]:
# mkdir bedbase_tutorial
cd bedbase_tutorial
export BEDBASE_DATA_PATH_HOST=`pwd`
export CODE=`pwd`

bash: cd: bedbase_tutorial: No such file or directory


Download some example BED files:

In [7]:
wget http://big.databio.org/example_data/bedbase_tutorial/bed_files.tar.gz     

--2023-08-11 08:10:02--  http://big.databio.org/example_data/bedbase_tutorial/bed_files.tar.gz
Resolving big.databio.org (big.databio.org)... 128.143.223.179
Connecting to big.databio.org (big.databio.org)|128.143.223.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44549692 (42M) [application/octet-stream]
Saving to: ‘bed_files.tar.gz’

bed_files.tar.gz    100%[===================>]  42.49M   303KB/s    in 95s     

2023-08-11 08:11:37 (458 KB/s) - ‘bed_files.tar.gz’ saved [44549692/44549692]



The downloaded files are compressed so we'll need to untar them:

In [8]:
tar -zxvf bed_files.tar.gz && mv bed_files files

bed_files/
bed_files/GSE105587_ENCFF018NNF_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSM2423312_ENCFF155HVK_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF617QGK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF316ASR_peaks_GRCh38.bed.gz
bed_files/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz
bed_files/GSM2827349_ENCFF196DNQ_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF553KIK_optimal_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE91663_ENCFF319TPR_conservative_idr_thresholded_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz
bed_files/GSM2827350_ENCFF928JXU_peaks_GRCh38.bed.gz
bed_files/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38.bed.gz


In [9]:
rm bed_files.tar.gz

Additionally, we'll download a matrix we need to provide if we wish to plot the tissue specificity of our set of genomic ranges:

Lastly, we'll download the core pipelines and tools needed to complete this tutorial: `bedmaker`, `bedqc`, `bedstat`, `bedbuncher` , `bedhost`, and `bedhost-ui`

In [5]:
pip install looper==1.5.1
pip install refgenie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 KB 734.8 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 724.9 kB/s eta 0:00:00 kB/s eta 0:00:01
  Attempting uninstall: pipestat
    Found existing installation: pipestat 0.5.0
    Uninstalling pipestat-0.5.0:
      Successfully uninstalled pipestat-0.5.0
  Attempting uninstall: looper
    Found existing installation: looper 1.5.0
    Uninstalling looper-1.5.0:
      Successfully uninstalled looper-1.5.0


In [11]:
git clone -b dev-bedboss git@github.com:databio/bedbase.git
pip install bedboss==0.1.0a2
# git clone -b validate_genome_assembly git@github.com:databio/bedbuncher
# git clone git@github.com:databio/bedembed
# git clone -b dev git@github.com:databio/bedhost
# git clone git@github.com:databio/bedhost-ui

Cloning into 'bedbase'...
remote: Enumerating objects: 664, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 664 (delta 188), reused 250 (delta 148), pack-reused 347
Receiving objects: 100% (664/664), 695.03 KiB | 386.00 KiB/s, done.
Resolving deltas: 100% (337/337), done.
  Using cached bbconf-0.4.0a1-py3-none-any.whl (11 kB)
  Using cached piper-0.13.2-py3-none-any.whl (72 kB)
  Attempting uninstall: piper
    Found existing installation: piper 0.12.3
    Uninstalling piper-0.12.3:
      Successfully uninstalled piper-0.12.3
  Attempting uninstall: bbconf
    Found existing installation: bbconf 0.4.0
    Uninstalling bbconf-0.4.0:
      Successfully uninstalled bbconf-0.4.0
  Attempting uninstall: bedboss
    Found existing installation: bedboss 0.1.0.dev2
    Uninstalling bedboss-0.1.0.dev2:
      Successfully uninstalled bedboss-0.1.0.dev2


### Let's install this packages!

    I have problems with bedtoBigBed script, so I am downloading it too, and seting in bedmaker path to this script :/

In [28]:
wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
chmod a+x bedToBigBed

--2023-08-11 07:51:37--  http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9632264 (9.2M)
Saving to: ‘bedToBigBed’

bedToBigBed         100%[===================>]   9.19M   740KB/s    in 18s     

2023-08-11 07:51:56 (524 KB/s) - ‘bedToBigBed’ saved [9632264/9632264]



In [6]:
pwd

/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial


Check if we have all tutorial files:

In [7]:
ls bedbase/tutorial_files

bedbase_configuration_compose_local.yaml  bedboss  scripts
bedbase_configuration_compose.yaml        PEPs


# 2. BEDBOSS: ALL TOGETHER

In [8]:
pip list | grep bbconf

bbconf                   0.4.0a1


In [9]:
pip list | grep bedboss

bedboss                  0.1.0a2


### Check and update config files

In [10]:
ls bedbase/tutorial_files/bedboss

bedboss_pep_config.yaml  looper_config_bedboss.yaml  sample_table.csv
config_db_local.yaml     pipeline_interface.yaml


Let's create additional metadata for our database:

In [11]:
cat bedbase/tutorial_files/bedboss/bedboss_pep_config.yaml

pep_version: 2.1.0
sample_table: sample_table.csv

sample_modifiers:
  append:
    input_file_path: INPUT
    output_folder: "$BEDBASE_DATA_PATH_HOST/outputs"
    narrowpeak: TRUE
    rfg_config_path: RFG
    bedbase_config: "$BEDBASE_DATA_PATH_HOST/bedbase/tutorial_files/bedboss/config_db_local.yaml"
    yaml_file: YAMLFILE
  derive:
    attributes: [input_file_path, rfg_config_path, yaml_file]
    sources:
      INPUT: "$BEDBASE_DATA_PATH_HOST/files/{file_name}"
      RFG: "$REFGENIE"
      YAMLFILE: "$BEDBASE_DATA_PATH_HOST/outputs/outputs/bedstat_output/bedstat_pipeline_logs/submission/{sample_name}_sample.yaml"
  imply:
    - if:
        antibody: [ H3K4me3, H3K27me3, H3K27ac, H3K9ac, H4K5ac, H3K4me, H3K36me3, H4K5ac, H3K9ac ]
      then:
        narrowpeak: FALSE


Config for local db and bedstat

In [12]:
cat bedbase/tutorial_files/bedboss/config_db_local.yaml

path:
  pipeline_output_path: $BEDBASE_DATA_PATH_HOST/outputs
  bedstat_dir: outputs/bedstat_output
  bedbuncher_dir: outputs/bedbuncher_output
  remote_url_base: null
database:
  host: $DB_HOST_URL
  port: $POSTGRES_PORT
  password: $POSTGRES_PASSWORD
  user: $POSTGRES_USER
  name: $POSTGRES_DB
  dialect: postgresql
  driver: psycopg2
server:
  host: 0.0.0.0
  port: 8080


looper for bedboss

In [13]:
cat bedbase/tutorial_files/bedboss//pipeline_interface.yaml

pipeline_name: BEDBOSS
pipeline_type: sample
pre_submit:
  python_functions:
    - looper.write_sample_yaml
command_template: >
  bedboss all
  --sample-name {sample.sample_name}
  --input-file {sample.input_file_path}
  --input-type {sample.format}
  --genome {sample.genome}
  --sample-yaml {sample.yaml_file}
  --output_folder {sample.output_folder}
  --narrowpeak {sample.narrowpeak}
  --rfg-config {sample.rfg_config_path}
  {% if sample.bedbase_config is defined %} --bedbase-config {sample.bedbase_config} {% endif %}
  {% if sample.chrom_sizes is defined %} --chrom-sizes {sample.chrom_sizes} {% endif %}
  {% if sample.open_signal_matrix is defined %} --open-signal-matrix {sample.open_signal_matrix} {% endif %}
  {% if sample.ensdb is defined %} --ensdb {sample.ensdb} {% endif %}
  {% if sample.fasta is defined %} --fasta {sample.fasta} {% endif %}
  --outfolder $BEDBASE_DATA_PATH_HOST/outputs/outputs/bedstat_output/bedstat_pipeline_logs


Looper config file:

In [16]:
ls bedbase/tutorial_files/bedboss

bedboss_pep_config.yaml  looper_config_bedboss.yaml  sample_table.csv
config_db_local.yaml     pipeline_interface.yaml


In [17]:
cat bedbase/tutorial_files/bedboss/looper_config_bedboss.yaml

pep_config: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/bedbase/tutorial_files/bedboss/bedboss_pep_config.yaml
output_dir: $BEDBASE_DATA_PATH_HOST/outputs/outputs/bedstat_output/bedstat_pipeline_logs

pipeline_interfaces:
  sample:  /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/bedbase/tutorial_files/bedboss//pipeline_interface.yaml


### Start DB (bedbase-postgres)

### Inititiate a local PostgreSQL instance

In addition to generate statistics and plots, `bedstat` inserts JSON formatted metadata into relational [PostgreSQL] database. 

If you don't have docker installed, you can install it with `sudo apt-get update && apt-get install docker-engine -y`.

Now, create a persistent volume to house PostgreSQL data:

In [35]:
docker volume create postgres-data

postgres-data


Spin up a `postgres` container. Provide required environment variables (need to match the settings in bedbase configuration file) and bind the created docker volume to `/var/lib/postgresql/data` path in the container:

In [36]:
docker run -d --name bedbase-postgres -p 5432:5432 -e POSTGRES_PASSWORD=bedbasepassword -e POSTGRES_USER=postgres -e POSTGRES_DB=postgres -v postgres-data:/var/lib/postgresql/data postgres

42ed2028444042c3ceef801c0828ce016dde87f1c0ac0d9494ffb6274374f262
docker: Error response from daemon: driver failed programming external connectivity on endpoint bedbase-postgres (fe853ffbf2fa584785686c319c5a657021a860dce6c9e81f67f5e805ef2133a0): Bind for 0.0.0.0:5432 failed: port is already allocated.


: 125

If environment variables are not initialized with function above, We have to initialize them manually 

In [29]:
export DB_HOST_URL=localhost
export POSTGRES_PORT=5432
export POSTGRES_PASSWORD=docker
export POSTGRES_USER=postgres
export POSTGRES_DB=pep-db

### RUN BEDBoss

Additionally, we have to initialize environment variable $REFGENIE - the path to the refgenie configuration file. If Refgenie is not initialize, we will have to initialize it localy. use `pip install --user refgenie` to install and add to the PATH with `export PATH=~/.local/bin:$PATH`

In [19]:
export REFGENIE='genome_config.yaml'
refgenie init -c $REFGENIE

Traceback (most recent call last):
  File "/home/bnt4me/virginia/venv/jupyter/bin/refgenie", line 8, in <module>
    sys.exit(main())
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/refgenie/cli.py", line 133, in main
    rgc.initialize_config_file(os.path.abspath(gencfg))
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/refgenconf/refgenconf.py", line 290, in initialize_config_file
    _write_fail_err("file exists")
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/refgenconf/refgenconf.py", line 281, in _write_fail_err
    raise OSError("Can't initialize, {}: {} ".format(reason, filepath))
OSError: Can't initialize, file exists: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/genome_config.yaml 


: 1

In [20]:
pwd

/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial


In [21]:
ls bedbase/tutorial_files/bedboss

bedboss_pep_config.yaml  looper_config_bedboss.yaml  sample_table.csv
config_db_local.yaml     pipeline_interface.yaml


##### Run bedboss

In [22]:
looper --version

looper 1.5.1



In [23]:
looper run --help

usage: looper run [-h] [-i] [-d] [-t S] [-x S] [-y S] [-f] [--divvy DIVCFG] [-p P] [-s S]
                  [-c K [K ...]] [-u X] [-n N] [--looper-config LOOPER_CONFIG]
                  [-S YAML [YAML ...]] [-P YAML [YAML ...]] [-l N] [-k N]
                  [--sel-attr ATTR] [--sel-excl [E ...] | --sel-incl [I ...]]
                  [-a A [A ...]]
                  [config_file]

Run or submit sample jobs.

positional arguments:
  config_file                        Project configuration file (YAML) or pephub registry
                                     path.

options:
  -h, --help                         show this help message and exit
  -i, --ignore-flags                 Ignore run status flags? Default=False
  -d, --dry-run                      Don't actually submit the jobs. Default=False
  -t S, --time-delay S               Time delay in seconds between job submissions
  -x S, --command-extra S            String to append to every command
  -y S, --command-extra-override S   S

In [56]:
pwd

/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial


In [24]:
$BEDBASE_DATA_PATH_HOST

bash: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial: Is a directory


: 126

In [28]:
looper run --looper-config ./bedbase/tutorial_files/bedboss/looper_config_bedboss.yaml --output-dir $BEDBASE_DATA_PATH_HOST/outputs/outputs/bedstat_output/bedstat_pipeline_logs

Looper version: 1.5.1
Command: run
Using default config. No config found in env var: ['DIVCFG']
Pipestat compatible: False
## [1 of 11] sample: bedbase_demo_db1; pipeline: BEDBOSS
Calling pre-submit function: looper.write_sample_yaml
Writing script to /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDBOSS_bedbase_demo_db1.sub
Job script (n=1; 0.00Gb): /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/submission/BEDBOSS_bedbase_demo_db1.sub
Compute node: bnt4me-Precision-5560
Start time: 2023-08-14 09:27:35
Using default config. No config found in env var: PIPESTAT_CONFIG
Config: None.
No schema supplied.
Initialize FileBackend
### Pipeline run code and environment:

*              Command:  `/home/bnt4me/virginia/venv/jupyter/bin/bedboss all --sample-name bedbase_demo_db1 --input-file /home/bnt4me/virginia/repos/

    self.db_engine_key = create_engine(self.db_url, echo=self.show_db_logs)
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlmodel/engine/create.py", line 139, in create_engine
    return _create_engine(url, **current_kwargs)  # type: ignore
  File "<string>", line 2, in create_engine
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlalchemy/util/deprecations.py", line 309, in warned
    return fn(*args, **kwargs)
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlalchemy/engine/create.py", line 518, in create_engine
    u = _url.make_url(url)
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlalchemy/engine/url.py", line 725, in make_url
    return _parse_rfc1738_args(name_or_url)
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlalchemy/engine/url.py", line 781, in _parse_rfc1738_args
    components["port"] = int(components["port"])
ValueError: invalid literal f

  return create_model(
Using default schema: /home/bnt4me/virginia/venv/jupyter/bin/pipestat_output_schema.yaml
Traceback (most recent call last):
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/pipestat/backends/dbbackend.py", line 689, in _engine
    return self.db_engine_key
AttributeError: 'DBBackend' object has no attribute 'db_engine_key'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/bnt4me/virginia/venv/jupyter/bin/bedboss", line 8, in <module>
    sys.exit(main())
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/bedboss/bedboss.py", line 180, in main
    run_all(pm=pm, **args_dict)
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/bedboss/bedboss.py", line 138, in run_all
    bedstat(
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/bedboss/bedstat/bedstat.py", line 103, in bedstat
    bbc = bbconf.BedBaseConf(confi

<pre>
</pre>
Command completed. Elapsed time: 0:00:00. Running peak memory: 0GB.  
  PID: 24374;	Command: zcat;	Return code: 0;	Memory used: 0.0GB


> `bash /home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/bedboss/bedqc/est_line.sh /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/bedmaker_logs/bedbase_demo_db3/_5zvvg7p `
File (/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/bedmaker_logs/bedbase_demo_db3/_5zvvg7p) has passed Quality Control!
Generating bigBed files for: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/files/GSE105977_ENCFF793SZW_conservative_idr_thresholded_peaks_GRCh38.bed.gz
Determining path to chrom.sizes asset via Refgenie.
Reading refgenie genome configuration file from file: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bed

* Records count: 2
* Sample name: DEFAULT_SAMPLE_NAME


----------------------------------------

Unused arguments: {'command': 'all', 'silent': False, 'verbosity': None, 'logdev': False}
Getting Open Signal Matrix file path...
output_bed = /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz
output_bigbed = /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bigbed_files
Got input type: bed
Converting /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/files/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz to BED format.
Target exists: `/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/GSE105977_ENCFF937CGY_peaks_GRCh38.bed.gz`  
Running bedqc...
Unused arguments: {}
Targ

...

File (/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/bedmaker_logs/bedbase_demo_db8/ny2pxb01) has passed Quality Control!
Generating bigBed files for: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/files/GSM2423312_ENCFF155HVK_peaks_GRCh38.bed.gz
Determining path to chrom.sizes asset via Refgenie.
Reading refgenie genome configuration file from file: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/genome_config.yaml
/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/alias/hg38/fasta/default/hg38.chrom.sizes
Determined path to chrom.sizes asset: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/alias/hg38/fasta/default/hg38.chrom.sizes
Config: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/bedbase/tutorial_files/bedboss/config_db_local.yaml.
Initiali

Got input type: bed
Converting /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/files/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz to BED format.
Target exists: `/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz`  
Running bedqc...
Unused arguments: {}
Target to produce: `/home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/bedmaker_logs/bedhost_demo_db9/h6i4w9_0`  

> `zcat /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/GSM2423313_ENCFF722AOG_peaks_GRCh38.bed.gz > /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/bed_files/bedmaker_logs/bedhost_demo_db9/h6i4w9_0` (2455

*  Pipeline started at:   (08-14 09:27:43) elapsed: 0.0 _TIME_

### Version log:

*       Python version:  3.10.12
*          Pypiper dir:  `/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/pypiper`
*      Pypiper version:  0.13.2
*         Pipeline dir:  `/home/bnt4me/virginia/venv/jupyter/bin`
*     Pipeline version:  0.1.0a2

### Arguments passed to pipeline:


### Initialized Pipestat Object:

* PipestatManager (bedboss-pipeline)
* Backend: File
*  - results: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/stats.yaml
*  - status: /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs
* Multiple Pipelines Allowed: True
* Pipeline name: bedboss-pipeline
* Pipeline type: sample
* Status Schema key: None
* Results formatter: default_formatter
* Results schema source: None
* Status schema source: None
* Records coun

Compute node: bnt4me-Precision-5560
Start time: 2023-08-14 09:27:43
Using default config. No config found in env var: PIPESTAT_CONFIG
Config: None.
No schema supplied.
Initialize FileBackend
### Pipeline run code and environment:

*              Command:  `/home/bnt4me/virginia/venv/jupyter/bin/bedboss all --sample-name bedbase_demo_db11 --input-file /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/files/GSM2827350_ENCFF928JXU_peaks_GRCh38.bed.gz --input-type bed --genome hg38 --sample-yaml /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs/outputs/bedstat_output/bedstat_pipeline_logs/submission/bedbase_demo_db11_sample.yaml --output_folder /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/outputs --narrowpeak True --rfg-config genome_config.yaml --bedbase-config /home/bnt4me/virginia/repos/bedbase_all/bedbase/docs_jupyter/bedbase_tutorial/bedbase/tutorial_files/bedboss/config_db_local.yaml --ou

  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/sqlalchemy/engine/url.py", line 781, in _parse_rfc1738_args
    components["port"] = int(components["port"])
ValueError: invalid literal for int() with base 10: '%24POSTGRES_PORT'
Starting cleanup: 1 files; 0 conditional files for cleanup

Cleaning up flagged intermediate files. . .

### Pipeline failed at:  (08-14 09:27:44) elapsed: 0.0 _TIME_

Total time: 0:00:00
Failure reason: Pipeline failure. See details above.
Exception ignored in atexit callback: <bound method PipelineManager._exit_handler of <pypiper.manager.PipelineManager object at 0x7fe2f62bc2b0>>
Traceback (most recent call last):
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/pypiper/manager.py", line 2191, in _exit_handler
    self.fail_pipeline(Exception("Pipeline failure. See details above."))
  File "/home/bnt4me/virginia/venv/jupyter/lib/python3.10/site-packages/pypiper/manager.py", line 2036, in fail_pipeline
    raise e

### ❗❗ If You have errors in bedstat requirements:

In [8]:
pip install -r bedstat/requirements.txt --user > requirements_log.txt

: 1

Install R dependencies

In [32]:
Rscript bedstat/scripts/installRdeps.R > R_deps.txt

In case there is an issue installing `GenomicDistributionsData`, try:
```
wget http://big.databio.org/GenomicDistributionsData/GenomicDistributionsData_0.0.2.tar.gz
Rscript -e 'install.packages("GenomicDistributionsData_0.0.2.tar.gz", type="source", repos=NULL)'
```

There's an additional dependency needed by `bedstat` if we wish to calculate and plot the GC content of our bedfiles. Depending on the genome assemblies of the files listed on a PEP, the appropriate BSgenome packages should be installed. The following is an example of how we can do so:

In [12]:
cat bedbase/tutorial_files/scripts/BSgenome_install.R

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("BSgenome.Hsapiens.UCSC.hg38.masked")

In [31]:
Rscript bedbase/tutorial_files/scripts/BSgenome_install.R > BSgenome.txt

We'll need to create a directory where we can store the stats and plots generated by `bedstat`. Additionally, we'll create a directory where we can store log and metadata files that we'll need later on.

## 4. BEDBUNCHER: Create bedsets and their respective statistics 

### Create a new PEP describing the bedset name and specific JSON query 

Now that we've processed several individual BED files, we'll turn to the next task: grouping them together into collections of BED files, which we call *bedsets*. For this, we use the `bedbuncher` pipeline, which produces outputs for each bedset, such as a bedset PEP, bedset-level statistics and plots, and an `IGD` database. To run `bedbuncher`, we will need another PEP describing each bedset. Though the annotation sheet below specifies attributes for one bedset, you can create as many as you wish using additional rows. For each bedset, you need to provide the query to retrieve certain collection BED files. 

The following example PEP shows the attributes we need to provide for each bedset and the config.yaml file that will grab the files needed to run `bedbuncher`:

In [31]:
cat bedbase/tutorial_files/PEPs/bedbuncher_query.csv

sample_name,bedset_name,genome,query,operator,query_val,bbconfig_name,bedbase_config
sample1,bedsetOver1kRegions,hg38,'regions_no',gt,"""1000""",bedbase_configuration_compose,source1
sample2,bedsetOver50GCContent,hg38,'gc_content',gt,"""0.5""",bedbase_configuration_compose,source1
sample3,bedsetUnder500MeanWidth,hg38,'mean_region_width',lt,"""500""",bedbase_configuration_compose,source1
sample4,bedsetTestSelectCellType,hg38,"""other::text~~:str_1 or other::text~~:str_2""","""str_1,str_2""","""%GM12878%,%HEK293%""",bedbase_configuration_compose,source1
sample5,bedsetTestSelectGenome,hg38,"""name=:name_1 or name=:name_2""","""name_1,name_2""","""GSE105587_ENCFF018NNF_conservative_idr_thresholded_peaks_GRCh38,GSE91663_ENCFF553KIK_optimal_idr_thresholded_peaks_GRCh38""",bedbase_configuration_compose,source1
sample6,bedsetTestCellType,hg38,"""other""",contains,"""""{\""cell_type\"":\ \""K562\""}""""",bedbase_configuration_compose,source1
sample7,bedsetTestSpace,hg38,"""other""",contains,"""

In [32]:
cat bedbase/tutorial_files/PEPs/bedbuncher_config.yaml

pep_version: 2.0.0
sample_table: bedbuncher_query.csv

looper:
    output_dir: $BEDBASE_DATA_PATH_HOST/outputs/bedbuncher_output/bedbuncher_pipeline_logs

sample_modifiers:
  append:
    pipeline_interfaces: $CODE/bedbuncher/pipeline_interface.yaml 
  derive:
    attributes: [bedbase_config]
    sources:
      source1: $CODE/bedbase/tutorial_files/{bbconfig_name}.yaml


Running `bedbuncher` with arguments defined in the example PEP above will result in a bedset with bedfiles that consist of at least 1000 regions.

###  Create outputs directory and install bedbuncher command line dependencies

We need a folder where we can store bedset related outputs. Though not required, we'll also create a directory where we can store the `bedbuncher` pipeline logs. 

In [33]:
mkdir -p outputs/bedbuncher_output/bedbuncher_pipeline_logs

One of the feats of `bedbuncher` includes [IGD](https://github.com/databio/IGD) database creation from the files in the bedset. `IGD` can be installed by cloning the repository from github, executing the make file to create the binary, and pointing the binary location with the `$PATH` environment variable. 

In [34]:
git clone git@github.com:databio/IGD
cd IGD
make > igd_make_log.txt 2>&1
cd ..

export PATH=$BEDBASE_DATA_PATH_HOST/IGD/bin/:$PATH

Cloning into 'IGD'...
remote: Enumerating objects: 1297, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 1297 (delta 35), reused 40 (delta 17), pack-reused 1230
Receiving objects: 100% (1297/1297), 949.45 KiB | 10.79 MiB/s, done.
Resolving deltas: 100% (804/804), done.


### Run bedbuncher using Looper 

Once we have cloned the `bedbuncher` repository, set our local Postgres cluster and created the `iGD` binary, we can run the pipeline by pointing `looper run` to the appropriate `PEP` config file. As mentioned earlier, if the path to the bedbase configuration file has been stored in the `$BEDBASE` environment variable, it's not neccesary to pass the `--bedbase-config` argument. 

In [36]:
looper run  bedbase/tutorial_files/PEPs/bedbuncher_config.yaml  --package local \
--command-extra="-R" > outputs/bedbuncher_output/bedbuncher_pipeline_logs/looper_logs.txt

Looper version: 1.3.1
Command: run
/home/bnt4me/.local/lib/python3.8/site-packages/divvy/compute.py:150: UserWarning: The '_file_path' property is deprecated and will be removed in a future release. Use ComputingConfiguration["__internal"]["_file_path"] instead.
  os.path.dirname(self._file_path),
/home/bnt4me/.local/lib/python3.8/site-packages/divvy/compute.py:58: UserWarning: The '_file_path' property is deprecated and will be removed in a future release. Use ComputingConfiguration["__internal"]["_file_path"] instead.
  self.config_file = self._file_path
Activating compute package 'local'
## [1 of 10] sample: sample1; pipeline: BEDBUNCHER
Writing script to /home/bnt4me/Virginia/bed_maker/bedbase_tutorial/outputs/bedbuncher_output/bedbuncher_pipeline_logs/submission/BEDBUNCHER_sample1.sub
Job script (n=1; 0.00Gb): /home/bnt4me/Virginia/bed_maker/bedbase_tutorial/outputs/bedbuncher_output/bedbuncher_pipeline_logs/submission/BEDBUNCHER_sample1.sub
## [2 of 10] sample: sample2; pipeline:

## 5. BEDEMBED: 

### bedembed_train: Uses the StarSpace method to embed the bed files and the meta data.

We need to install [StarSpace](https://github.com/facebookresearch/StarSpace) first.  

In [ ]:
mkdir -p bedembed/tools

We need to install [Boost](http://www.boost.org/) library and specify the path of boost library in makefile in order to run StarSpace.

In [ ]:
wget https://boostorg.jfrog.io/artifactory/main/release/1.78.0/source/boost_1_78_0.zip
unzip boost_1_78_0.zip
sudo mv boost_1_78_0 /usr/local/bin
cd /usr/local/bin/boost_1_78_0
./bootstrap.sh
./b2

In order to build StarSpace on Mac OS or Linux, use the following:

In [ ]:
cd $BEDBASE_DATA_PATH_HOST/bedembed/tools
git clone https://github.com/facebookresearch/Starspace.git
cd Starspace
make
make embed_doc

We need a folder where we can store bedembed related outputs. Though not required, we'll also create a directory where we can store the bedembed pipeline logs.

In [ ]:
mkdir -p outputs/bedembed_output/bedembed_pipeline_logs

In [ ]:
path_starspace=$BEDBASE_DATA_PATH_HOST'/bedembed/tools/Starspace/starspace'
path_meta=$BEDBASE_DATA_PATH_HOST'/bedbase/tutorial_files/PEPs/bedstat_annotation_sheet.csv'
# download Universe file from rivanna
path_universe=$BEDBASE_DATA_PATH_HOST'/tiles1000.hg19.bed'
path_output=$BEDBASE_DATA_PATH_HOST'/outputs/bedembed_output/'
assembly='hg38'
path_data=$BEDBASE_DATA_PATH_HOST'/bed_files/'
labels="exp_protocol,cell_type,tissue,antibody,treatment"
no_files=10
start_line=0
dim=50
epochs=20
learning_rate=0.001

python ./bedembed/pipeline/bedembed_train.py -star $path_starspace -i $path_data -g $assembly -meta $path_meta -univ $path_universe \
-l $labels -nof $no_files -o $path_output -startline $start_line -dim $dim -epochs $epochs -lr $learning_rate

### bedembed_test: calculate the distances between file labels and trained search terms

### Get a PEP describing the bedfiles to process 

We'll use the standard [PEP](http://pep.databio.org) format for the annotation, which consists of 1) a sample table (.csv) that annotates the files, and 2) a project config.yaml file that points to the sample annotation sheet. The config file also has other components, such as derived attributes, that in this case point to the bedfiles to be processed. Here is the PEP config file for this example project:

In [1]:
cat bedbase/tutorial_files/PEPs/bedembed_test_config.yaml

bedembed_version: 0.0.0
sample_table: bedstat_annotation_sheet.csv

looper:
  output-dir: $BEDBASE_DATA_PATH_HOST/outputs/bedembed_output/bedembed_pipeline_logs 
sample_modifiers:
  append:
    bedbase_config: $BEDBASE_DATA_PATH_HOST/bedbase/tutorial_files/bedbase_configuration_compose.yaml
    pipeline_interfaces: $BEDBASE_DATA_PATH_HOST/bedembed/pipeline_interface_test.yaml
    universe: /project/shefflab/data/StarSpace/universe/universe_tilelen1000.bed
    input_file_path: INPUT
    output_file_path: $BEDBASE_DATA_PATH_HOST/outputs/bedembed_output
    yaml_file: SAMPLE_YAML
  derive:
    attributes: [yaml_file, input_file_path]
    sources:
      INPUT: "/project/shefflab/data/encode/{file_name}"
      SAMPLE_YAML: "$BEDBASE_DATA_PATH_HOST/outputs/bedembed_output/bedembed_pipeline_logs/submission/{sample_name}_sample.yaml"


### Run bedembed using Looper 

Once we have cloned the `bedembed` repository, set our local postgres cluster, we can run the pipeline by pointing `looper run` to the appropriate `PEP` config file. As mentioned earlier, if the path to the bedbase configuration file is provided, the calculated distances will report to the postgres database, if not it will save as a csv file in the `output_file_path`

In [ ]:
looper run bedbase/tutorial_files/PEPs/bedembed_test_config.yaml --package local

## 5. BEDHOST:  Serve BED files and API to explore pipeline outputs

The last part of the tutorial consists on running a local instance of `bedhost` (a REST API for `bedstat` and `bedbuncher` produced outputs) in order to explore plots, statistics and download pipeline outputs. 
To run `bedhost`, frist use `bedhost-ui` to built the bedhost user interface with React.

In [38]:
cd bedhost-ui
# Install node modules defined in package.json
npm install 
# Build the app for production to the ./build folder
npm run build
# copy the contents of the ./build directory to bedhost/bedhost/static/bedhost-ui
cp -avr ./build ../bedhost/bedhost/static/bedhost-ui

cd ..

To run `bedhost`, we'll pip install the package from the previously cloned repository:

In [39]:
pip install bedhost/. --user > bedhost_log.txt

To start `bedhost`, we simply need to run the following command passing the location of the bedbase configuration file to the `-c` flag.  

In [ ]:
bedhost serve -c  $BEDBASE_DATA_PATH_HOST/bedbase/tutorial_files/bedbase_configuration_compose.yaml

Serving data for columns: ['md5sum']
Serving data for columns: ['md5sum']
Generating GraphQL schema
running bedhost app
INFO:     Started server process [648505]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:47532 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:47532 - "GET /ui/static/css/2.fa6c921b.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:47534 - "GET /ui/static/css/main.4620a2c9.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:47536 - "GET /ui/static/js/2.b0639060.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:47534 - "GET /ui/static/js/main.56118e82.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:47536 - "GET /api/bed/all/data/count HTTP/1.1" 200 OK
[(None,), ({'alias': 'hg38', 'digest': '2230c535660fb4774114bfa966a62f823fdb6d21acf138d4'},)]
INFO:     127.0.0.1:47532 - "GET /api/bed/genomes HTTP/1.1" 200 OK
INFO:     127.0.0.1:47534 - "GET /api/versions HTT

If we have stored the path to the bedbase config in the environment variable `$BEDBASE` (suggested), it's not neccesary to use said flag. 

In [ ]:
bedhost serve 

The `bedhost` API can be opened in the url [http://0.0.0.0:8000](http://0.0.0.0:8000). We can now explore the plots and statistics generated by the `bedstat` and `bedbuncher` pipelines.

## or optionally run BEDHOST using containers

Alternatively, you can run the application inside a container.

For that we'll use [docker compose](https://docs.docker.com/compose/), a tool that makes running multi-contaier Docker applications possible. The `docker-compose.yaml` file defines two services: 
- `fastapi-api`: runs the fastAPI server 
- `postgres-db`: runs the PostgeSQL database used by the server


In [24]:
cd $BEDBASE_DATA_PATH_HOST

Use the `BEDBASE_DATA_PATH_HOST` environment variable to point to the host directory with the pipeline results that will be mounted in the container as a volume. 

The environment variables are passed to the container via `.env` file, which the `docker-compose.yaml` points to for each service. Additionally, you can just export the environment variables before issuing the `docker-compose` command.
When you set the same environment variable in multiple files, here’s the priority used by Compose to choose which value to use:

1. Compose file
2. Shell environment variables
3. Environment file
4. Dockerfile
4. Variable is not defined

In [26]:
cd bedhost; docker-compose up

Pulling postgres-db (postgres:)...
latest: Pulling from library/postgres
Digest: sha256:8f7c3c9b61d82a4a021da5d9618faf056633e089302a726d619fa467c73609e4
Status: Downloaded newer image for postgres:latest
Recreating postgreSQL-bedbase ... 
Recreating fastAPI-bedbase    ... mdone
Attaching to postgreSQL-bedbase, fastAPI-bedbase
postgreSQL-bedbase | 
postgreSQL-bedbase | PostgreSQL Database directory appears to contain a database; Skipping initialization
postgreSQL-bedbase | 
postgreSQL-bedbase | 2020-11-02 23:10:28.883 UTC [1] LOG:  starting PostgreSQL 13.0 (Debian 13.0-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit
postgreSQL-bedbase | 2020-11-02 23:10:28.885 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
postgreSQL-bedbase | 2020-11-02 23:10:28.885 UTC [1] LOG:  listening on IPv6 address "::", port 5432
postgreSQL-bedbase | 2020-11-02 23:10:28.891 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
postgreSQL-bed